In [1]:
#####Bayesian Personalized Ranking#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize

In [2]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no, pattern):
    if pattern==1:
        z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
        Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
        return z_id, Z
    z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
    return z_id

In [3]:
####データの生成####
##データの設定
k = 10
f = 150
hh = 4000
item = 2500
Lambda = np.random.gamma(30, 1/0.3, hh)
pt = np.random.poisson(Lambda, hh)
hhpt = np.sum(pt)
k_vec = np.repeat(1.0, k)

In [4]:
##IDとインデックスの設定
#IDの設定
no = np.arange(hhpt)
d_id = np.repeat(np.arange(hh), pt)
pt_id = np.array(list(itertools.chain(*[np.array(range(pt[i]), dtype="int") for i in range(hh)])))

#インデックスの設定
d_list = [i for i in range(hh)]
for i in range(hh):
    d_list[i] = np.array(np.where(d_id==i)[0], dtype="int") 

In [5]:
##アイテム集合を生成
#トピック割当を生成
topic = 50
Gamma = np.random.gamma(5.5, 1/0.7, topic)
phi_topic = np.random.dirichlet(np.repeat(0.1, item), topic)
theta_topic = np.random.dirichlet(np.repeat(0.3, topic), hh)

#多項分布からアイテム集合を生成
max_item = 25
set_list = [i for i in range(hhpt)]
set_dt = np.array(np.full((hhpt, max_item), item), dtype="int16")
item_no = np.arange(item)
m = np.repeat(0, hhpt)
for i in range(hh):
    if i%1000==0:
        print(i)
    target_no = no[d_list[i]]
    z = np.dot(np.random.multinomial(1, theta_topic[i, ], pt[i]), np.arange(topic))
    m_vec = np.random.poisson(Gamma[z], pt[i])
    m_vec[m_vec < 2] = 2; m_vec[m_vec > max_item] = max_item
    m[d_list[i]] = m_vec
    for j in range(pt[i]):
        while True:
            sets = np.random.multinomial(1, phi_topic[z[j], ], m_vec[j])
            if np.max(np.sum(sets, axis=0))==1:
                sets_id = np.dot(sets, item_no)
                set_list[target_no[j]] = sets_id
                set_dt[target_no[j], np.arange(m_vec[j])] = sets_id
                break
                
#リストの変換
sets_id = np.array(list(itertools.chain(*[set_list[i] for i in range(hhpt)])))
no_long = np.repeat(np.arange(hhpt), m)
d_long = np.repeat(d_id, m)
N = d_long.shape[0]
No = np.arange(N)

0
1000
2000
3000


In [6]:
#インデックスを定義
no_list = [i for i in range(hhpt)]
d_list = [i for i in range(hh)]
for i in range(hhpt):
    if i==0:
        no_list[i] = np.arange(m[i])
    else:
        no_list[i] = np.max(no_list[i-1]) + np.arange(m[i]) + 1
for i in range(hh):
    d_list[i] = np.array(np.where(d_long==i)[0], dtype="int")

In [7]:
##応答変数を生成
#高次元特徴行列を生成
F = np.array(np.abs(np.random.normal(0, 0.3, item*f).reshape(f, item)), dtype="float32")

#モデルパラメータを生成
Sigma = np.array([1.0])
beta_u = np.random.normal(0, 0.5, hh)
beta_v = np.random.normal(0, 0.75, item)
theta_u = np.random.normal(0, 0.6, k*hh).reshape(hh, k)
theta_v = np.random.normal(0, 0.75, k*item).reshape(item, k)
omega = np.random.normal(0, 0.25, f*k).reshape(k, f)
betat_u = beta_u.copy(); betat_v = beta_v.copy()
thetat_u = theta_u.copy(); thetat_v = theta_v.copy(); omegat = omega.copy()

#効用関数の期待値を定義
uv1 = np.dot(theta_u[d_long, ] * theta_v[sets_id, ], k_vec)
uv2 = np.dot(theta_u[d_long, ] * np.dot(omega, F[:, sets_id]).T, k_vec)
mu = beta_u[d_long] + beta_v[sets_id] + uv1 + uv2

#選択アイテムを生成
U = mu + np.random.normal(0, Sigma, N)
item_id = np.repeat(0, hhpt)
target_index = np.repeat(0, hhpt)
for i in range(hhpt):
    index1 = no_list[i]; index2 = np.argmax(U[index1])
    item_id[i] = sets_id[index1][index2]
    target_index[i] = No[index1][index2]
item_long = np.repeat(item_id, m)

#選択集合から選択されたアイテムを除外
get_index = np.array(np.where(item_long!=sets_id)[0], dtype="int")
n = m - 1 
no_long = np.repeat(np.arange(hhpt), n)
d_long = d_long[get_index]
item_long = item_long[get_index]
sets_id = sets_id[get_index]
N = get_index.shape[0]
No = np.arange(N)

#新しい高次元特徴行列を定義
intercept = np.repeat(1.0, N)[:, np.newaxis]
F1 = F[:, item_long]
F2 = F[:, sets_id]

In [8]:
##新しいインデックスを定義
#ユーザーインデックスを定義
no_list = [i for i in range(hhpt)]
d_list = [i for i in range(hh)]
for i in range(hhpt):
    if i==0:
        no_list[i] = np.arange(n[i])
    else:
        no_list[i] = np.max(no_list[i-1]) + np.arange(n[i]) + 1
for i in range(hh):
    d_list[i] = np.array(np.where(d_long==i)[0], dtype="int")
    
#アイテムインデックスを定義
item_list1 = [i for i in range(item)]; item_list2 = [i for i in range(item)]
item_n1 = np.repeat(0, item); item_n2 = np.repeat(0, item)
for i in range(item):
    item_list1[i] = np.array(np.where(item_long==i)[0], dtype="int")
    item_list2[i] = np.array(np.where(sets_id==i)[0], dtype="int")
    item_n1[i] = item_list1[i].shape[0]
    item_n2[i] = item_list2[i].shape[0]

In [9]:
####Bayesian Personalized Rankingを推定####
##パラメータ推定のための関数を定義
#対数事後分布の和を定義
def log_posterior(mu, beta_u, beta_v, theta_u, theta_v, omega, tau_u, tau_v, inv_Cov_u, inv_Cov_v, inv_Cov_g, k_vec):
    #ロジットモデルの対数尤度
    logit_exp = np.exp(mu)
    LLho = np.sum(np.log(logit_exp / (1 + logit_exp)))

    #モデルパラメータの対数事前分布
    Prior_u1 = np.sum(-0.5 * (np.power(beta_u, 2) / tau_u))
    Prior_v1 = np.sum(-0.5 * (np.power(beta_v, 2) / tau_v))
    Prior_u2 = np.sum(-0.5 * np.dot(np.dot(theta_u, inv_Cov_u) * theta_u, k_vec))
    Prior_v2 = np.sum(-0.5 * np.dot(np.dot(theta_v, inv_Cov_v) * theta_v, k_vec))
    Prior_g = np.sum(-0.5 * np.dot(np.dot(omega.T, inv_Cov_g) * (omega.T), k_vec))
    Prior = Prior_u1 + Prior_v1 + Prior_u2 + Prior_v2 + Prior_g

    #対数事後分布の和
    Posterior = LLho + Prior
    return Posterior

#ユーザーパラメータの勾配を定義
def dlogit_u(beta_u, theta_u, tau_u, inv_Cov_u, mu, intercept, theta_item, theta_sets, omega, F1, F2):
    #ロジットの確率を定義
    logit_exp = np.exp(-mu)
    Prob = (logit_exp / (1 + logit_exp))[:, np.newaxis]

    #勾配ベクトルを定義
    x = np.hstack((intercept, (theta_item - theta_sets) + np.dot(omega, F1 - F2).T))
    dlogit = Prob * x

    #事前分布の勾配を定義
    theta = np.hstack((beta_u[:, np.newaxis], theta_u))
    inv_Cov = np.diag(np.append(1/tau_u, np.diag(inv_Cov_u)))
    dmvn = -np.dot(inv_Cov, theta.T).T
    return dlogit, dmvn

#アイテムパラメータの勾配を定義
def dlogit_v(beta_v, theta_v, tau_v, inv_Cov_v, mu, intercept, theta_long):
    #ロジットの確率を定義
    logit_exp = np.exp(-mu)
    Prob = (logit_exp / (1 + logit_exp))[:, np.newaxis]

    #勾配ベクトルを定義
    x = np.hstack((intercept, theta_long))
    dlogit1 = Prob * x
    dlogit2 = Prob * (-x)

    #事前分布の勾配を定義
    theta = np.hstack((beta_v[:, np.newaxis], theta_v))
    inv_Cov = np.diag(np.append(1/tau_v, np.diag(inv_Cov_v)))
    dmvn = -np.dot(inv_Cov, theta.T).T
    return dlogit1, dlogit2, dmvn

#縮小ランクパラメータの勾配を定義
def dlogit_g(omega, inv_Cov_g, mu, theta_long, F1, F2):
    #ロジットの確率を定義
    logit_exp = np.exp(-mu)
    Prob = (logit_exp / (1 + logit_exp))[:, np.newaxis]

    #勾配ベクトルを定義
    dlogit = np.dot((Prob * theta_long).T, (F1 - F2).T)
    dmvn = -np.dot(inv_Cov_g, omega)
    return dlogit, dmvn

In [10]:
##アルゴリズムの設定
#確率的勾配法のパラメータを設定
eta_u = 0.0025
eta_v = 0.0025
eta_g = 0.0005

#アルゴリズムの収束判定
iters = 0
rp = 200   #最大繰り返し数
LL1 = -1000000000   #対数尤度の初期値
dl = 100   #EMアルゴリズムの対数尤度の差の初期値
tol = 1.0

In [11]:
#正則化パラメータを定義
tau_u = np.array([0.5])
tau_v = np.array([0.5])
Cov_u = np.diag(np.repeat(0.2, k))
Cov_v = np.diag(np.repeat(0.2, k))
Cov_g = np.diag(np.repeat(0.1, k))
inv_Cov_u = np.linalg.inv(Cov_u)
inv_Cov_v = np.linalg.inv(Cov_v)
inv_Cov_g = np.linalg.inv(Cov_g)

In [12]:
##パラメータの真値
#モデルパラメータの真値
dl = 100   
beta_u = betat_u.copy()
beta_v = betat_v.copy()
omega = omegat.copy()
theta_u = thetat_u.copy()
theta_v = thetat_v.copy()

#モデルの期待値を真値
beta_long = beta_u[d_long]; beta_item = beta_v[item_long]; beta_sets = beta_v[sets_id]
theta_long = theta_u[d_long, ]; theta_item = theta_v[item_long, ]; theta_sets = theta_v[sets_id, ] 
omega_f1 = np.dot(omega, F1).T; omega_f2 = np.dot(omega, F2).T
uv11 = np.dot(theta_long * theta_item, k_vec)
uv12 = np.dot(theta_long * theta_sets, k_vec)
uv21 = np.dot(theta_long * omega_f1, k_vec)
uv22 = np.dot(theta_long * omega_f2, k_vec)
mu1 = beta_long + beta_item + uv11 + uv21
mu2 = beta_long + beta_sets + uv12 + uv22
mu = mu1 - mu2
mut = mu.copy()

#対数尤度の初期値
logit_exp = np.exp(mu)
LL1 = np.sum(np.log(logit_exp / (1 + logit_exp)))

In [13]:
##パラメータの初期値
#モデルパラメータの初期値
dl = 100
beta_u = np.random.normal(0, 0.25, hh)
beta_v = np.random.normal(0, 0.25, item)
theta_u = np.random.normal(0, 0.25, k*hh).reshape(hh, k)
theta_v = np.random.normal(0, 0.25, k*item).reshape(item, k)
omega = np.random.normal(0, 0.2, f*k).reshape(k, f)

#モデルの期待値を初期値
beta_long = beta_u[d_long]; beta_item = beta_v[item_long]; beta_sets = beta_v[sets_id]
theta_long = theta_u[d_long, ]; theta_item = theta_v[item_long, ]; theta_sets = theta_v[sets_id, ] 
omega_f1 = np.dot(omega, F1).T; omega_f2 = np.dot(omega, F2).T
uv11 = np.dot(theta_long * theta_item, k_vec)
uv12 = np.dot(theta_long * theta_sets, k_vec)
uv21 = np.dot(theta_long * omega_f1, k_vec)
uv22 = np.dot(theta_long * omega_f2, k_vec)
mu1 = beta_long + beta_item + uv11 + uv21
mu2 = beta_long + beta_sets + uv12 + uv22
mu = mu1 - mu2

#対数尤度の初期値
logit_exp = np.exp(mu)
LL1 = np.sum(np.log(logit_exp / (1 + logit_exp)))

In [14]:
##対数尤度の基準値
#真値での対数尤度
logit_exp = np.exp(mut)
LLbest = np.sum(np.log(logit_exp / (1 + logit_exp)))
print(LLbest)

-587018.4131837866


In [15]:
#####MAP推定でパラメータを推定####
while abs(dl) >= tol:

    ##ユーザーパラメータを更新
    #ユーザーパラメータの勾配を定義
    dlogit, dmvn = dlogit_u(beta_u, theta_u, tau_u, inv_Cov_u, mu, intercept, theta_item, theta_sets, omega, F1, F2)

    #勾配法でパラメータを更新
    new_theta = np.zeros((hh, k+1))
    for i in range(hh):
        old_theta = np.append(beta_u[i], theta_u[i, ])
        new_theta[i, ] = old_theta + eta_u*(np.sum(dlogit[d_list[i], ], axis=0) + dmvn[i, ])

    #パラメータを更新
    beta_u = new_theta[:, 0]; theta_u = new_theta[:, 1:]
    beta_long = beta_u[d_long]; theta_long = theta_u[d_long, ]

    #モデルの期待値を更新
    uv11 = np.dot(theta_long * theta_item, k_vec)
    uv12 = np.dot(theta_long * theta_sets, k_vec)
    uv21 = np.dot(theta_long * omega_f1, k_vec)
    uv22 = np.dot(theta_long * omega_f2, k_vec)
    mu1 = beta_long + beta_item + uv11 + uv21
    mu2 = beta_long + beta_sets + uv12 + uv22
    mu = mu1 - mu2


    ##アイテムパラメータを更新
    #アイテムパラメータの勾配を定義
    dlogit1, dlogit2, dmvn = dlogit_v(beta_v, theta_v, tau_v, inv_Cov_v, mu, intercept, theta_long)

    #勾配法でパラメータを更新
    new_theta = np.zeros((item, k+1))
    for i in range(item):
        old_theta = np.append(beta_v[i], theta_v[i, ])
        dlogit_sum1 = np.sum(dlogit1[item_list1[i], ], axis=0)
        dlogit_sum2 = np.sum(dlogit2[item_list2[i], ], axis=0)
        new_theta[i, ] = old_theta + eta_v*(dlogit_sum1 + dlogit_sum2 + dmvn[i, ])

    #パラメータを更新
    beta_v = new_theta[:, 0]; theta_v = new_theta[:, 1:]
    beta_item = beta_v[item_long]; beta_sets = beta_v[sets_id]
    theta_item = theta_v[item_long, ]; theta_sets = theta_v[sets_id, ] 

    #モデルの期待値を更新
    uv11 = np.dot(theta_long * theta_item, k_vec)
    uv12 = np.dot(theta_long * theta_sets, k_vec)
    mu1 = beta_long + beta_item + uv11 + uv21
    mu2 = beta_long + beta_sets + uv12 + uv22
    mu = mu1 - mu2


    ##縮小ランクパラメータを更新
    #勾配法でパラメータを更新 
    dlogit, dmvn = dlogit_g(omega, inv_Cov_g, mu, theta_long, F1, F2)   #縮小ランクパラメータの勾配
    omega += eta_g * (dlogit + dmvn)

    #モデルの期待値を定義
    omega_f1 = np.dot(omega, F1).T; omega_f2 = np.dot(omega, F2).T
    uv21 = np.dot(theta_long * omega_f1, k_vec)
    uv22 = np.dot(theta_long * omega_f2, k_vec)
    mu1 = beta_long + beta_item + uv11 + uv21
    mu2 = beta_long + beta_sets + uv12 + uv22
    mu = mu1 - mu2


    ##対数尤度の更新とアルゴリズムの収束判定
    #対数尤度を更新
    logit_exp = np.exp(mu)
    LL = np.sum(np.log(logit_exp / (1 + logit_exp)))

    #アルゴリズムの収束判定
    iters = iters + 1
    dl = LL - LL1
    LL1 = LL
    print(LL)

-1781819.389072703
-1555541.0474901332
-1396421.7751569988
-1158343.81945168
-1025541.280440739
-921187.1295887661
-848955.5523801168
-794647.4220049236
-752987.3013175355
-723834.2006210908
-705480.3495451513
-693055.7091389154
-691862.9532708144
-688656.0392305036
-675909.7783389443
-658963.6428474499
-647334.0811131597
-641173.0557707074
-636145.0412167185
-627247.7830074575
-621680.1699257017
-615543.345230549
-609989.9787303816
-600080.2793595382
-590506.6005769174
-581042.448409845
-574249.6844863194
-568034.4598000456
-563748.7398425775
-559910.0030380664
-557628.8339454823
-555534.8508878575
-554415.6653469376
-552935.1316793691
-551854.4916400177
-550199.3658647598
-548849.9376793437
-547081.6187910648
-545643.7180328728
-543899.3870696113
-542480.7665940376
-540842.0637604642
-539538.2656849525
-538093.2471940921
-536984.2176661567
-535783.6583276343
-534896.9638020206
-533938.2251720512
-533257.5914070645
-532503.8557248142
-531991.0856400833
-531390.956773709
-530999.361564